In [1]:
#pip install matplotlib 

In [2]:
#Importing Libraries 
import tkinter as tk
import numpy as np
import random
import math
import time
import matplotlib.pyplot as plt 

In [3]:
#Defining Coordinates 
Coor1=70,70,100,100
Coor2=180,70,210,100
Coor3=290,70,320,100
Coor4=400,70,430,100

Coor5=70,180,100,210
Coor6=180,180,210,210
Coor7=290,180,320,210
Coor8=400,180,430,210


Coor10=180,290,210,320
Coor11=290,290,320,320
Coor12=400,290,430,320

list5=[Coor1, Coor2, Coor3, Coor4, Coor5, Coor6, Coor7, Coor8, Coor10, Coor11, Coor12]

In [4]:
#Defining the electrons' coordinates 
electron=random.choice(list5)
electron2=random.choice(list5)

if electron==Coor12:
    WIN_STATE=(2, 3)
    rectangle_electron=360,250,470,360
if electron==Coor11:
    WIN_STATE=(2, 2)
    rectangle_electron=250,250,360,360
if electron==Coor10:
    WIN_STATE=(2, 1)
    rectangle_electron=140,250,250,360

if electron==Coor8:
    WIN_STATE=(1, 3)
    rectangle_electron=360,140,470,250
if electron==Coor7:
    WIN_STATE=(1, 2)
    rectangle_electron=250,140,360,250
if electron==Coor6:
    WIN_STATE=(1, 1)
    rectangle_electron=140,140,250,250
if electron==Coor5:
    WIN_STATE=(1, 0)
    rectangle_electron=30,140,140,250
if electron==Coor4:
    WIN_STATE=(0, 3)
    rectangle_electron=360,30,470,140
if electron==Coor3:
    WIN_STATE=(0, 2)
    rectangle_electron=250,30,360,140
if electron==Coor2:
    WIN_STATE=(0, 1)
    rectangle_electron=140,30,250,140
if electron==Coor1:
    WIN_STATE=(0, 0)
    rectangle_electron=30,30,140,140
list6, list9=[], []


if electron2==Coor11:
    WIN_STATE2=(2, 2)
    rectangle_electron2=250,250,360,360
if electron2==Coor10:
    WIN_STATE2=(2, 1)
    rectangle_electron2=140,250,250,360
if electron2==Coor12:
    WIN_STATE2=(2, 3)
    rectangle_electron2=360,250,470,360
if electron2==Coor8:
    WIN_STATE2=(1, 3)
    rectangle_electron2=360,140,470,250
if electron2==Coor7:
    WIN_STATE2=(1, 2)
    rectangle_electron2=250,140,360,250
if electron2==Coor6:
    WIN_STATE2=(1, 1)
    rectangle_electron2=140,140,250,250
if electron2==Coor5:
    WIN_STATE2=(1, 0)
    rectangle_electron2=30,140,140,250
if electron2==Coor4:
    WIN_STATE2=(0, 3)
    rectangle_electron2=360,30,470,140
if electron2==Coor3:
    WIN_STATE2=(0, 2)
    rectangle_electron2=250,30,360,140
if electron2==Coor2:
    WIN_STATE2=(0, 1)
    rectangle_electron2=140,30,250,140
if electron2==Coor1:
    WIN_STATE2=(0, 0)
    rectangle_electron2=30,30,140,140

In [5]:
#set the rows and columns length
BOARD_ROWS = 3
BOARD_COLS = 4

#initalise start, win and lose states
START = (2, 3)

In [6]:
#class state defines the board and decides reward, end and next position
class State:
    def __init__(self, state=START):
        #initalise the state to start and end to false
        self.state = state
        self.isEnd = False        

    def getReward(self):
        #give the rewards for each state -5 for loss, +1 for win, -1 for others
        if self.state == WIN_STATE or self.state==WIN_STATE2:
            return 1       
        
        else:
            return -2.5

    def isEndFunc(self):
        #set state to end if win/loss
        if (self.state == WIN_STATE or self.state==WIN_STATE2):
            self.isEnd = True


    def nxtPosition(self, action):     
        #set the positions from current action - up, down, left, right
        if action == 0:                
            nxtState = (self.state[0] - 1, self.state[1]) #up             
        elif action == 1:
            nxtState = (self.state[0] + 1, self.state[1]) #down
        elif action == 2:
            nxtState = (self.state[0], self.state[1] - 1) #left
        else:
            nxtState = (self.state[0], self.state[1] + 1) #right


        #check if next state is possible
        if (nxtState[0] >= 0) and ((nxtState[0] <= WIN_STATE[0]) or (nxtState[0] <= WIN_STATE2[0])):
            if (nxtState[1] >= 0) and ((nxtState[1] <= WIN_STATE[1]) or (nxtState[1] <= WIN_STATE2[1])):   
                    #if possible change to next state                
                    return nxtState 
        #Return current state if outside grid     
        return self.state 




In [7]:
#class agent to implement reinforcement learning through grid  
class Agent:

    def __init__(self):
        #inialise states and actions 
        self.states = []
        self.actions = [0,1,2,3]    # up, down, left, right
        self.State = State()
        #set the learning and greedy values
        self.alpha = 0.6
        self.gamma = 0.9
        self.epsilon = 0.0005
        self.isEnd = self.State.isEnd

        # array to retain reward values for plot
        self.plot_reward = []
        
        #initalise Q values as a dictionary for current and new
        self.Q = {}
        self.new_Q = {}
        #initalise rewards to 0
        self.rewards = 0
        
        #initalise all Q values across the board to 0, print these values
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                for k in range(len(self.actions)):
                    self.Q[(i, j, k)] =0
                    self.new_Q[(i, j, k)] = 0
        
        print(self.Q)
        
    

    #method to choose action with Epsilon greedy policy, and move to next state
    def Action(self):
        #random value vs epsilon
        rnd = random.random()
        #set arbitraty low value to compare with Q values to find max
        mx_nxt_reward =-10
        action = None
        
        #9/10 find max Q value over actions 
        if(rnd >self.epsilon) :
            #iterate through actions, find Q  value and choose best 
            for k in self.actions:
                
                i,j = self.State.state
                
                nxt_reward = self.Q[(i,j, k)]
                
                if nxt_reward >= mx_nxt_reward:
                    action = k
                    mx_nxt_reward = nxt_reward
                    
        #else choose random action
        else:
            action = np.random.choice(self.actions)
        
        #select the next state based on action chosen
        position = self.State.nxtPosition(action)
        return position,action
    
    
    #Q-learning Algorithm
    def Q_Learning(self,episodes):
        x = 0
        #iterate through best path for each episode
        while(x < episodes):
            #check if state is end
            if self.isEnd:
                #get current rewrard and add to array for plot
                reward = self.State.getReward()
                self.rewards += reward
                self.plot_reward.append(self.rewards)
                
                #get state, assign reward to each Q_value in state
                i,j = self.State.state
                for a in self.actions:
                    self.new_Q[(i,j,a)] = round(reward,3)
                    
                #reset state
                self.State = State()
                self.isEnd = self.State.isEnd
                
                #set rewards to zero and iterate to next episode
                self.rewards = 0.8
                x+=1
            else:
                #set to arbitrary low value to compare net state actions
                mx_nxt_value = -10
                #get current state, next state, action and current reward
                next_state, action = self.Action()
                i,j = self.State.state
                reward = self.State.getReward()
                #add reward to rewards for plot
                self.rewards +=reward
                
                #iterate through actions to find max Q value for action based on next state action
                for a in self.actions:
                    nxtStateAction = (next_state[0], next_state[1], a)
                    q_value = (1-self.alpha)*self.Q[(i,j,action)] + self.alpha*(reward + self.gamma*self.Q[nxtStateAction])
                
                    #find largest Q value
                    if q_value >= mx_nxt_value:
                        mx_nxt_value = q_value
                
                #next state is now current state, check if end state
                self.State = State(state=next_state)
                self.State.isEndFunc()
                self.isEnd = self.State.isEnd
                #update Q values with max Q value for next state
                self.new_Q[(i,j,action)] = round(mx_nxt_value,3)
            
            #copy new Q values to Q table
            self.Q = self.new_Q.copy()
            
        #print final Q table output
        
       
        
    #plot the reward vs episodes
    def plot(self,episodes):
        plt.plot(self.plot_reward)
        plt.show()
        
        
    #iterate through the board and find largest Q value in each, print output
    def showValues(self):
        for i in range(0, BOARD_ROWS):
            print('-----------------------------------------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                mx_nxt_value = -10
                for a in self.actions:
                    nxt_value = self.Q[(i,j,a)]
                    if nxt_value >= mx_nxt_value:
                        mx_nxt_value = nxt_value
                out += str(mx_nxt_value).ljust(6) + ' | '
                list9.append(float(str(mx_nxt_value).ljust(6)))
                if float(str(mx_nxt_value).ljust(6))>=0:
                    list6.append(float(str(mx_nxt_value).ljust(6)))

            print(out)
        print('-----------------------------------------------')

In [8]:
#Defining Boltzmann's policy class 
list10=[]
class BoltzmannPolicy:
    def __init__(self, actions=None, tau=500, tau_decay=False, decay_factor_tau=0.1):
        assert tau >= 0
        assert bool == type(tau_decay)
        self.actions = actions
        self.tau = tau
        self.tau_decay = tau_decay
        self.decay_factor_tau = decay_factor_tau

    def compute_action(self, q_list):
        if(self.actions is None):
            self.actions = [i for i in range(len(q_list))]
        assert len(self.actions) == len(q_list)
        probs = []
        for q in q_list:
            num = math.exp(q/self.tau)
            exp_values_for_dem = []
            for q_t in q_list:
                if q_t !=0: 
                    if q_t ==1: 
                        exp_values_for_dem.append(1)
                    else: 
                        exp_values_for_dem.append(math.exp(q_t/self.tau))

                else: 
                    exp_values_for_dem.append(0)
            list11=exp_values_for_dem
            Coord1=list11[0]
            Coord2=list11[1]
            Coord3=list11[2]
            Coord4=list11[3]
            Coord5=list11[4]
            Coord6=list11[5]
            Coord7=list11[6]
            Coord8=list11[7]
            Coord9=list11[8]
            Coord10=list11[9]
            Coord11=list11[10]
            Coord12=list11[11]
            
            if "1" in {Coord10, Coord11, Coord12}: 
                list10.append("right")
                list10.append("right")
            elif "1" in {Coord1, Coord5}: 
                list10.append("up")
                list10.append("up")
            else: 
                if Coord2==1: 
                    if (Coord5+Coord1)>(Coord10+Coord6):
                        list10.append("up")
                        list10.append("up")
                        list10.append("right")
                    else: 
                        list10.append("right")
                        list10.append("up")
                        list10.append("up")
                elif Coord3==1: 
                    if (Coord5+Coord1)>(Coord10+Coord6):
                        list10.append("up")
                        list10.append("up")
                        list10.append("right")
                        list10.append("right")
                    elif (Coord10+Coord11+Coord7)>(Coord5+Coord1+Coord2):
                        list10.append("right")
                        list10.append("right")
                        list10.append("up")
                        list10.append("up")
                    else: 
                        list10.append("right")
                        list10.append("up")
                        list10.append("up")
                        list10.append("right")
                elif Coord4==1: 
                    if max((Coord10+Coord11+Coord12+Coord8),(Coord5+Coord1+Coord2+Coord3),(Coord10+Coord6+Coord2+Coord3), (Coord10+Coord11+Coord7+Coord3))==(Coord10+Coord11+Coord12+Coord8):
                        list10.append("right")
                        list10.append("right")
                        list10.append("right")
                        list10.append("up")
                        list10.append("up")
                    elif max((Coord10+Coord11+Coord12+Coord8),(Coord5+Coord1+Coord2+Coord3),(Coord10+Coord6+Coord2+Coord3), (Coord10+Coord11+Coord7+Coord3))==(Coord5+Coord1+Coord2+Coord3):
                        list10.append("up")
                        list10.append("up")
                        list10.append("right")
                        list10.append("right")
                        list10.append("right")
                    elif max((Coord10+Coord11+Coord12+Coord8),(Coord5+Coord1+Coord2+Coord3),(Coord10+Coord6+Coord2+Coord3), (Coord10+Coord11+Coord7+Coord3))==(Coord10+Coord6+Coord2+Coord3):
                        list10.append("right")
                        list10.append("up")
                        list10.append("up")
                        list10.append("right")
                        list10.append("right")
                    else: 
                        list10.append("right")
                        list10.append("right")
                        list10.append("up")
                        list10.append("up")
                        list10.append("right")
                elif Coord8==1: 
                    if max((Coord10+Coord11+Coord12),(Coord5+Coord6+Coord7),(Coord10+Coord6+Coord7), (Coord10+Coord11+Coord7))==(Coord10+Coord11+Coord12):
                        list10.append("right")
                        list10.append("right")
                        list10.append("right")
                        list10.append("up")
                    elif max((Coord10+Coord11+Coord12),(Coord5+Coord6+Coord7),(Coord10+Coord6+Coord7), (Coord10+Coord11+Coord7))==(Coord5+Coord6+Coord7):
                        list10.append("up")
                        list10.append("right")
                        list10.append("right")
                        list10.append("right")
                    elif max((Coord10+Coord11+Coord12),(Coord5+Coord6+Coord7),(Coord10+Coord6+Coord7), (Coord10+Coord11+Coord7))==(Coord10+Coord6+Coord7):
                        list10.append("right")
                        list10.append("up")
                        list10.append("right")
                        list10.append("right")
                    else: 
                        list10.append("right")
                        list10.append("right")
                        list10.append("up")
                        list10.append("right")
                elif Coord7==1: 
                    if max((Coord10+Coord11),(Coord5+Coord6),(Coord10+Coord6))==(Coord10+Coord11):
                        list10.append("right")
                        list10.append("right")
                        list10.append("up")
                    elif max((Coord10+Coord11),(Coord5+Coord6),(Coord10+Coord6))==(Coord5+Coord6):
                        list10.append("up")
                        list10.append("right")
                        list10.append("right")
                    else: 
                        list10.append("right")
                        list10.append("up")
                        list10.append("right")
                elif Coord6==1: 
                    if Coord5>Coord10:
                        list10.append("up")
                        list10.append("right")
                    else: 
                        list10.append("right")
                        list10.append("up")

        return list10


In [9]:
#Playing 
list2, list3, list4 = [], [], []
NN_episodes=1000
if __name__ == "__main__":
    episodes = 10000
    ag = Agent()
    ag.Q_Learning(episodes)
    ag.showValues()
    p=BoltzmannPolicy()
    q=p.compute_action(list9)
    print(q)
for i in range(0,NN_episodes): 
    x=[1,2]
    xx=random.choice(x)
    list3.append(xx)
    if NN_episodes-10<i<NN_episodes:
        list2.append(xx) 
    if 0<i<NN_episodes:
        list4.append(xx)
    i+=1
print(list2, list3, list4)

{(0, 0, 0): 0, (0, 0, 1): 0, (0, 0, 2): 0, (0, 0, 3): 0, (0, 1, 0): 0, (0, 1, 1): 0, (0, 1, 2): 0, (0, 1, 3): 0, (0, 2, 0): 0, (0, 2, 1): 0, (0, 2, 2): 0, (0, 2, 3): 0, (0, 3, 0): 0, (0, 3, 1): 0, (0, 3, 2): 0, (0, 3, 3): 0, (1, 0, 0): 0, (1, 0, 1): 0, (1, 0, 2): 0, (1, 0, 3): 0, (1, 1, 0): 0, (1, 1, 1): 0, (1, 1, 2): 0, (1, 1, 3): 0, (1, 2, 0): 0, (1, 2, 1): 0, (1, 2, 2): 0, (1, 2, 3): 0, (1, 3, 0): 0, (1, 3, 1): 0, (1, 3, 2): 0, (1, 3, 3): 0, (2, 0, 0): 0, (2, 0, 1): 0, (2, 0, 2): 0, (2, 0, 3): 0, (2, 1, 0): 0, (2, 1, 1): 0, (2, 1, 2): 0, (2, 1, 3): 0, (2, 2, 0): 0, (2, 2, 1): 0, (2, 2, 2): 0, (2, 2, 3): 0, (2, 3, 0): 0, (2, 3, 1): 0, (2, 3, 2): 0, (2, 3, 3): 0}
-----------------------------------------------
| 0      | 0      | 0      | 0      | 
-----------------------------------------------
| 0      | 0      | 0      | 1      | 
-----------------------------------------------
| 0      | 0      | 0      | -1.6   | 
-----------------------------------------------
['right', 'right',

In [11]:
#Gridworld class
class Gridworld():
    def Grid(self):

        self.root = tk
        self.panvas = self.root.Canvas(bg='black',height=400, width=500)
        self.panvas.pack()


        Q_values = self.panvas.create_rectangle(30, 30, 470, 360, fill='grey')
        goal= self.panvas.create_rectangle(rectangle_electron,fill='green')
        electroon=self.panvas.create_oval(electron, fill='black')
        goal2= self.panvas.create_rectangle(rectangle_electron2, fill="rosybrown")
        electroon2=self.panvas.create_oval(electron2, fill='black')
        

        v1 = self.panvas.create_line(140, 30, 140, 360, fill='white', width=2)
        v2 = self.panvas.create_line(250, 30, 250, 360, fill='white', width=2)
        v3 = self.panvas.create_line(360, 30, 360, 360, fill='white', width=2)
        v4 = self.panvas.create_line(140, 30, 140, 360, fill='white', width=2)
        v5 = self.panvas.create_line(140, 30, 140, 360, fill='white', width=2)
        v6 = self.panvas.create_line(30, 140, 470, 140, fill='white', width=2)
        v7 = self.panvas.create_line(30, 250, 470, 250, fill='white', width=2)

        top = self.panvas.create_line(28, 30, 472, 30, fill='white', width=5)
        left = self.panvas.create_line(30, 30, 30, 360, fill='white', width=5)
        bottom = self.panvas.create_line(28, 360, 473, 360, fill='white', width=5)
        right = self.panvas.create_line(470, 28, 470, 362, fill='white', width=5)

        self.agent = self.panvas.create_oval(70,290,100,320, fill='cyan')

        self.panvas.pack()

        # self.root.mainloop()

    def reset(self):

        self.panvas.delete(self.agent)
        self.agent = self.panvas.create_oval(70, 290, 100, 320, fill='cyan')
        self.panvas.update()
        time.sleep(0.5)

    def move_agent(self):
        self.Grid()
        self.panvas.update()
        time.sleep(1)

        for i in range(1, 100):
               
            for j in range(0, len(list10)):
                self.panvas.update()
                s = self.panvas.coords(self.agent)
                move=list10[j]
                
                #Defining the way the agent plays 
                if move == "up":

                    if s[0] == 70:
                    
                        if (s[1]==electron[1] and s[2]==electron[2]) or (s[1]==electron2[1] and s[2]==electron2[2]): 
                                self.reset()
                        if s[1]>70:
                            move_agent = np.array([0, -110])
                            self.panvas.move(self.agent, move_agent[0], move_agent[1])
                            s = self.panvas.coords(self.agent)
                            self.panvas.update()
                            time.sleep(0.2)
                            if (s[1]==electron[1] and s[2]==electron[2]) or (s[1]==electron2[1] and s[2]==electron2[2]): 
                                self.reset()
                        

                    if s[0] == 180:
                        if (s[1]==electron[1] and s[2]==electron[2]) or (s[1]==electron2[1] and s[2]==electron2[2]): 
                                self.reset()

                        if s[1]>70: 
                            move_agent = np.array([0, -110])
                            self.panvas.move(self.agent, move_agent[0], move_agent[1])
                            s = self.panvas.coords(self.agent)
                            self.panvas.update()
                            time.sleep(0.2)
                            if (s[1]==electron[1] and s[2]==electron[2]) or (s[1]==electron2[1] and s[2]==electron2[2]): 
                                self.reset()

                    if s[0] == 290:
                        if (s[1]==electron[1] and s[2]==electron[2]) or (s[1]==electron2[1] and s[2]==electron2[2]): 
                                self.reset()

                        if s[1]>70:
                            move_agent = np.array([0, -110])
                            self.panvas.move(self.agent, move_agent[0], move_agent[1])
                            s = self.panvas.coords(self.agent)
                            self.panvas.update()
                            time.sleep(0.2)
                            if (s[1]==electron[1] and s[2]==electron[2]) or (s[1]==electron2[1] and s[2]==electron2[2]): 
                                self.reset()

                    if s[0] == 400:
                        if (s[1]==electron[1] and s[2]==electron[2]) or (s[1]==electron2[1] and s[2]==electron2[2]): 
                                self.reset()

                        if s[1]>70:
                            move_agent = np.array([0, -110])
                            self.panvas.move(self.agent, move_agent[0], move_agent[1])
                            s = self.panvas.coords(self.agent)
                            self.panvas.update()
                            time.sleep(0.2)
                            if (s[1]==electron[1] and s[2]==electron[2]) or (s[1]==electron2[1] and s[2]==electron2[2]): 
                                self.reset()


                if move == "right":

                    if s[1] == 290:
                        if (s[0]==electron[0] and s[3]==electron[3]) or (s[0]==electron2[0] and s[3]==electron2[3]): 
                            self.reset()
                        if s[0]<291:
                            move_agent = np.array([110, 0])
                            self.panvas.move(self.agent, move_agent[0], move_agent[1])
                            s = self.panvas.coords(self.agent)
                            self.panvas.update()
                            time.sleep(0.2)
                            if (s[0]==electron[0] and s[3]==electron[3]) or (s[0]==electron2[0] and s[3]==electron2[3]): 
                                self.reset()

                    if s[1] == 70:
                        if (s[0]==electron[0] and s[3]==electron[3]) or (s[0]==electron2[0] and s[3]==electron2[3]): 
                                self.reset()

                        if s[0]<291:

                            move_agent = np.array([110, 0])
                            self.panvas.move(self.agent, move_agent[0], move_agent[1])
                            s = self.panvas.coords(self.agent)
                            self.panvas.update()
                            time.sleep(0.2)
                            if (s[0]==electron[0] and s[3]==electron[3]) or (s[0]==electron2[0] and s[3]==electron2[3]): 
                                self.reset()

                    if s[1] == 180:

                        if s[0]<291:
                            if (s[0]==electron[0] and s[3]==electron[3]) or (s[0]==electron2[0] and s[3]==electron2[3]): 
                                self.reset()

                            move_agent = np.array([110, 0])
                            self.panvas.move(self.agent, move_agent[0], move_agent[1])
                            s = self.panvas.coords(self.agent)
                            self.panvas.update()
                            time.sleep(0.2)
                            if (s[0]==electron[0] and s[3]==electron[3]) or (s[0]==electron2[0] and s[3]==electron2[3]): 
                                self.reset()
                        

                if move == "left":

                    if s[1] == 290:
                        if (s[0]==electron[0] and s[3]==electron[3]) or (s[0]==electron2[0] and s[3]==electron2[3]): 
                            self.reset()

                        if s[0]>70:
                            move_agent = np.array([-110, 0])
                            self.panvas.move(self.agent, move_agent[0], move_agent[1])
                            s = self.panvas.coords(self.agent)
                            self.panvas.update()
                            time.sleep(0.2)
                            if (s[0]==electron[0] and s[3]==electron[3]) or (s[0]==electron2[0] and s[3]==electron2[3]): 
                                self.reset()

                    if s[1] == 70:
                        if (s[0]==electron[0] and s[3]==electron[3]) or (s[0]==electron2[0] and s[3]==electron2[3]): 
                            self.reset()

                        if s[0]>70:
                            move_agent = np.array([-110, 0])
                            self.panvas.move(self.agent, move_agent[0], move_agent[1])
                            s = self.panvas.coords(self.agent)
                            self.panvas.update()
                            time.sleep(0.2)
                            if (s[0]==electron[0] and s[3]==electron[3]) or (s[0]==electron2[0] and s[3]==electron2[3]): 
                                self.reset()
                
                    if s[1] == 180:
                        if (s[0]==electron[0] and s[3]==electron[3]) or (s[0]==electron2[0] and s[3]==electron2[3]): 
                                self.reset()

                        if s[0]>70:
                            move_agent = np.array([-110, 0])
                            self.panvas.move(self.agent, move_agent[0], move_agent[1])
                            s = self.panvas.coords(self.agent)
                            self.panvas.update()
                            time.sleep(0.2)
                            if (s[0]==electron[0] and s[3]==electron[3]) or (s[0]==electron2[0] and s[3]==electron2[3]): 
                                self.reset()


                if move == "down":

                    if s[0] == 70:
                        if (s[1]==electron[1] and s[2]==electron[2]) or (s[1]==electron2[1] and s[2]==electron2[2]): 
                                self.reset()

                        if s[1]<290:
                            move_agent = np.array([0, 110])
                            self.panvas.move(self.agent, move_agent[0], move_agent[1])
                            s = self.panvas.coords(self.agent)
                            self.panvas.update()
                            time.sleep(0.2)
                            if (s[1]==electron[1] and s[2]==electron[2]) or (s[1]==electron2[1] and s[2]==electron2[2]): 
                                self.reset()

                    if s[0] == 290:
                        if (s[1]==electron[1] and s[2]==electron[2]) or (s[1]==electron2[1] and s[2]==electron2[2]): 
                            self.reset()

                        if s[1]<290:
                            move_agent = np.array([0, 110])
                            self.panvas.move(self.agent, move_agent[0], move_agent[1])
                            s = self.panvas.coords(self.agent)
                            self.panvas.update()
                            time.sleep(0.2)
                            if (s[1]==electron[1] and s[2]==electron[2]) or (s[1]==electron2[1] and s[2]==electron2[2]): 
                                self.reset()
                
                    if s[0] == 180:
                    
                        if (s[1]==electron[1] and s[2]==electron[2]) or (s[1]==electron2[1] and s[2]==electron2[2]): 
                                self.reset()

                        if s[1]<290:
                            move_agent = np.array([0, 110])
                            self.panvas.move(self.agent, move_agent[0], move_agent[1])
                            s = self.panvas.coords(self.agent)
                            self.panvas.update()
                            time.sleep(0.2)
                            if (s[1]==electron[1] and s[2]==electron[2]) or (s[1]==electron2[1] and s[2]==electron2[2]): 
                                self.reset()
                
                    if s[0] == 400:
                        if (s[1]==electron[1] and s[2]==electron[2]) or (s[1]==electron2[1] and s[2]==electron2[2]): 
                            self.reset()

                        if s[1]<290: 
                            move_agent = np.array([0, 110])
                            self.panvas.move(self.agent, move_agent[0], move_agent[1])
                            s = self.panvas.coords(self.agent)
                            self.panvas.update()
                            time.sleep(0.2)
                            if (s[1]==electron[1] and s[2]==electron[2]) or (s[1]==electron2[1] and s[2]==electron2[2]): 
                                self.reset()
                j+=1
        self.root.mainloop()

data = Gridworld()

# print(data.Grid())
print(data.move_agent())

TclError: invalid command name ".!canvas"